# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [75]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm
from sklearn.metrics import f1_score

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [2]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [3]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [4]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

aclImdb_v1.tar.gz:   0%|          | 164k/84.1M [00:00<01:00, 1.39MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 55.3MB/s]


In [5]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [56]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                        dropout=dropout, bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        # hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [57]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [62]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [63]:
model = model.to(device)

In [64]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [158]:
def train(model, iterator, optimizer, criterion, epoch):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [67]:
def evaluate(model, iterator, criterion, epoch):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
        pbar.set_description(f"Epoch {epoch}")
        for it, batch in pbar:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [72]:
import numpy as np

min_loss = np.inf

cur_patience = 0
max_grad_norm=0

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

for epoch in range(1, max_epochs + 1):
    train_loss, train_acc = train(model, train_iter, optimizer, criterion, epoch)
    val_loss, val_acc = evaluate(model, valid_iter, criterion, epoch)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

	Train Loss: 0.063 | Train Acc: 97.72%
	 Val. Loss: 0.600 |  Val. Acc: 85.24%
Epoch: 1, Training Loss: 0.06342668533172241, Validation Loss: 0.600048054009676


	Train Loss: 0.035 | Train Acc: 98.80%
	 Val. Loss: 0.671 |  Val. Acc: 84.98%
Epoch: 2, Training Loss: 0.035089453706521216, Validation Loss: 0.6714727956864793


	Train Loss: 0.019 | Train Acc: 99.42%
	 Val. Loss: 0.707 |  Val. Acc: 85.43%
Epoch: 3, Training Loss: 0.018865396309241205, Validation Loss: 0.7074873379226458


<All keys matched successfully>

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [135]:
def evaluate_f1(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            y_true = batch.label
            y_pred = torch.round(torch.sigmoid(predictions))
            acc = f1_score(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [78]:
test_loss, test_f1 = evaluate_f1(model, test_iter, criterion)

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [79]:
test_f1

0.2568334398976982

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очент хороший фильм" или "невероятноя скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [125]:
try:
  del train_iter
  del val_iter
  del test_iter
except:
  pass

In [126]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [127]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [90]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv2d(in_channels = 1, out_channels = out_channels, 
                                kernel_size = (kernel_sizes[0], dim))
        self.conv_1 = nn.Conv2d(in_channels = 1, out_channels = out_channels, 
                                kernel_size = (kernel_sizes[1], dim))
        self.conv_2 = nn.Conv2d(in_channels = 1, out_channels = out_channels, 
                                kernel_size = (kernel_sizes[2], dim))
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.unsqueeze(1)
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [91]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [92]:
model.to(device)

CNN(
  (embedding): Embedding(202652, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(1, 1))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(1, 1))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(1, 1))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [93]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [94]:
max_epochs = 30

Обучите!

In [160]:
def train(model, iterator, optimizer, criterion, epoch):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        optimizer.zero_grad()
        text = batch.text
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, epoch):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
        pbar.set_description(f"Epoch {epoch}")
        for it, batch in pbar:
            text = batch.text
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [132]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss, train_acc = train(model, train_iter, opt, loss_func, epoch)
    val_loss, val_acc = evaluate(model, val_iter, loss_func, epoch)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

	Train Loss: 0.411 | Train Acc: 81.36%
	 Val. Loss: 0.380 |  Val. Acc: 83.54%
Epoch: 1, Training Loss: 0.41063238016880343, Validation Loss: 0.3801631391048431


	Train Loss: 0.347 | Train Acc: 84.67%
	 Val. Loss: 0.361 |  Val. Acc: 84.30%
Epoch: 2, Training Loss: 0.3467515862553659, Validation Loss: 0.3606381545464198


	Train Loss: 0.285 | Train Acc: 88.08%
	 Val. Loss: 0.340 |  Val. Acc: 85.05%
Epoch: 3, Training Loss: 0.28452155711877086, Validation Loss: 0.33997310400009156


	Train Loss: 0.227 | Train Acc: 90.63%
	 Val. Loss: 0.335 |  Val. Acc: 85.32%
Epoch: 4, Training Loss: 0.22702104179528507, Validation Loss: 0.3351679886380831


	Train Loss: 0.172 | Train Acc: 93.35%
	 Val. Loss: 0.341 |  Val. Acc: 85.48%
Epoch: 5, Training Loss: 0.17241594467284907, Validation Loss: 0.34070452600717543


	Train Loss: 0.120 | Train Acc: 95.43%
	 Val. Loss: 0.349 |  Val. Acc: 85.41%
Epoch: 6, Training Loss: 0.12020256631348254, Validation Loss: 0.3485237583518028


<All keys matched successfully>

In [141]:
def evaluate_f1(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, batch.label)
            y_true = batch.label
            y_pred = torch.round(torch.sigmoid(predictions))
            acc = f1_score(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [142]:
test_loss, test_f1 = evaluate_f1(model, test_iter, loss_func)

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [143]:
test_f1

0.46885627706837

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [144]:
!pip install -q captum

     |████████████████████████████████| 1.4MB 5.3MB/s 


In [145]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [146]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [147]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.99 ) , delta:  tensor([0.0004], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([1.2967e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.30 ) , delta:  tensor([2.8339e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.27 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.60 ) , delta:  tensor([3.6622e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [148]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,1.46,It was a fantastic performance ! pad
pos,neg (0.01),pos,1.35,Best film ever pad pad pad pad
pos,neg (0.30),pos,1.18,Such a great show! pad pad pad
neg,neg (0.00),pos,0.27,It was a horrible movie pad pad
neg,neg (0.27),pos,0.93,I've never watched something as bad pad
neg,pos (0.60),pos,1.57,It is a disgusting movie! pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
TEXT.build_vocab(trn, vectors="glove.6B.300d")
LABEL.build_vocab(trn)
word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

In [154]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [155]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight.data.copy_(word_embeddings)

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

Вы знаете, что делать.

In [161]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss, train_acc = train(model, train_iter, opt, loss_func, epoch)
    val_loss, val_acc = evaluate(model, val_iter, loss_func, epoch)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

	Train Loss: 0.509 | Train Acc: 73.80%
	 Val. Loss: 0.362 |  Val. Acc: 84.62%
Epoch: 1, Training Loss: 0.5088811645107548, Validation Loss: 0.36228974064191183


	Train Loss: 0.316 | Train Acc: 86.69%
	 Val. Loss: 0.315 |  Val. Acc: 86.93%
Epoch: 2, Training Loss: 0.31596502237511376, Validation Loss: 0.3148831566174825


	Train Loss: 0.181 | Train Acc: 93.52%
	 Val. Loss: 0.307 |  Val. Acc: 87.39%
Epoch: 3, Training Loss: 0.1811308127141347, Validation Loss: 0.30671336154143014


	Train Loss: 0.083 | Train Acc: 97.58%
	 Val. Loss: 0.319 |  Val. Acc: 87.62%
Epoch: 4, Training Loss: 0.08342992666646512, Validation Loss: 0.3189982463916143


	Train Loss: 0.033 | Train Acc: 99.27%
	 Val. Loss: 0.349 |  Val. Acc: 87.66%
Epoch: 5, Training Loss: 0.03267255595402561, Validation Loss: 0.3488353212674459


<All keys matched successfully>

In [162]:
test_loss, test_f1 = evaluate_f1(model, test_iter, loss_func)

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [163]:
test_f1

0.4633422023516216

Проверим насколько все хорошо!

In [164]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([4.5898e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.19 ) , delta:  tensor([4.7125e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.85 ) , delta:  tensor([6.6726e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([3.8013e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.40 ) , delta:  tensor([3.4186e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([2.0626e-05], device='cuda:0', dtype=torch.float64)


In [165]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.68,It was a fantastic performance ! pad
pos,neg (0.19),pos,-0.00,Best film ever pad pad pad pad
pos,pos (0.85),pos,1.58,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.84,It was a horrible movie pad pad
neg,neg (0.40),pos,0.85,I've never watched something as bad pad
neg,neg (0.00),pos,-0.87,It is a disgusting movie! pad pad
